PDF 파일을 변환해서 original 문장 단위로 개행하여 txt 파일로 저장한 후 이를 S3에 업로드 한다.

In [1]:
import boto3
import re
import os
from pdfConverter import convert_pdf_to_txt
from pdfConverter import trim_txt
from txt2sent import extract_sents
import aws
from timeLimit import time_limit
from txt2sent import parts
from delete_reference import del_ref
from nltk import sent_tokenize

상수

In [2]:
rewrite = True  # 덮어쓰기
folder = re.compile('pdfDatas/data2/.+[.]pdf')  # Files to download from S3

In [3]:
# 폴더 생성
if not os.path.exists('pdfData'):
    os.makedirs('pdfData')
if not os.path.exists('txtData'):
    os.makedirs('txtData')
if not os.path.exists('sents'):
    os.makedirs('sents')

In [ ]:
pattern = re.compile('\(cid:[0-9]+\)')

for obj in aws.bucket.objects.all():
    try:
        # 우리가 원하는 pdf 파일이면
        if folder.match(obj.key) is not None:
            pdf_file_dir = 'pdfData/' + obj.key.split('/')[-1]
            txt_file_dir = pdf_file_dir.replace('pdf', 'txt')

            # s3에 변환된 문장이 이미 있는지 확인
            if aws.exists(txt_file_dir) and not rewrite:
                continue

            # pdf file이 local에 존재하는지 확인하고 없으면 다운로드
            if not os.path.exists(pdf_file_dir):
                aws.bucket.download_file(obj.key, pdf_file_dir)

            # pdf2txt 변환
            if not os.path.exists(txt_file_dir) or rewrite:
                text = convert_pdf_to_txt(pdf_file_dir)
                text = trim_txt(text)
                text = del_ref(text)
                text = pattern.sub('', text)
                sents = sent_tokenize(text)
                with open(txt_file_dir, 'w', encoding='UTF-8') as f:
                    for sent in sents:
                        sent = sent.replace('\n', ' ').replace('  ', ' ')
                        f.write(sent + '\n')

            # S3에 쓰기
            aws.s3.Object(aws.bucket_name, txt_file_dir).put(Body=open(txt_file_dir, 'rb'))
            # print(txt_file_dir + ' uploaded.')

    except Exception as e:
        print(e)